In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2021-ai-w6-p1/sample_submission.csv
/kaggle/input/2021-ai-w6-p1/pca_test.csv
/kaggle/input/2021-ai-w6-p1/pca_train.csv
/kaggle/input/2021-ai-w6-p1/train_label.csv


In [2]:
import pandas as pd
import numpy as np

x=pd.read_csv("/kaggle/input/2021-ai-w6-p1/pca_train.csv")
y=pd.read_csv("/kaggle/input/2021-ai-w6-p1/train_label.csv")
test=pd.read_csv("/kaggle/input/2021-ai-w6-p1/pca_test.csv")
submit=pd.read_csv("/kaggle/input/2021-ai-w6-p1/sample_submission.csv")

In [3]:
import torch
seed=777

torch.manual_seed(seed)

device='cuda' if torch.cuda.is_available() else 'cpu'

if device=='cuda':
  torch.cuda.manual_seed_all(seed)

In [4]:
y=y["label"]

In [5]:
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()
scaler.fit(x)
x=scaler.transform(x)
test=scaler.transform(test)

In [6]:
x=torch.FloatTensor(np.array(x))
y=torch.LongTensor(np.array(y))
test=torch.FloatTensor(np.array(test))

In [7]:
linear1=torch.nn.Linear(256,512)
linear2=torch.nn.Linear(512,1024)
linear3=torch.nn.Linear(1024,512)
linear4=torch.nn.Linear(512,198)

relu=torch.nn.ReLU()

dropout=torch.nn.Dropout(p=0.1)

In [8]:
torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)
torch.nn.init.xavier_uniform_(linear4.weight)

Parameter containing:
tensor([[-0.0139,  0.0101,  0.0797,  ..., -0.0294, -0.0765, -0.0173],
        [ 0.0220,  0.0674, -0.0853,  ..., -0.0389, -0.0498, -0.0406],
        [-0.0662,  0.0576,  0.0888,  ...,  0.0327, -0.0042,  0.0757],
        ...,
        [-0.0359, -0.0795,  0.0537,  ...,  0.0035,  0.0360, -0.0258],
        [-0.0030, -0.0303,  0.0256,  ..., -0.0699, -0.0785, -0.0285],
        [-0.0605, -0.0254, -0.0350,  ..., -0.0361,  0.0375,  0.0328]],
       requires_grad=True)

In [9]:
model=torch.nn.Sequential(linear1,relu,dropout,linear2,relu,dropout,linear3,relu,dropout,linear4).to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)
loss=torch.nn.CrossEntropyLoss().to(device)

In [10]:
model.train()
for epoch in range(1000):
  optimizer.zero_grad()
  h=model(x.to(device))
  cost=loss(h,y.to(device))
  cost.backward()
  optimizer.step()
  if epoch%10==0:
    print(epoch,cost.item())

0 6.842648506164551
10 3.090667724609375
20 2.250969886779785
30 1.915178656578064
40 1.6464240550994873
50 1.436258316040039
60 1.2810765504837036
70 1.1464179754257202
80 1.024838924407959
90 0.9056775569915771
100 0.7969476580619812
110 0.6889356970787048
120 0.5788769125938416
130 0.4918728768825531
140 0.4104114770889282
150 0.34055042266845703
160 0.28665947914123535
170 0.2432357519865036
180 0.20446455478668213
190 0.1775447130203247
200 0.1535743623971939
210 0.1317545473575592
220 0.11990995705127716
230 0.10965798050165176
240 0.09848568588495255
250 0.09067561477422714
260 0.08134220540523529
270 0.07598709315061569
280 0.07430875301361084
290 0.06952005624771118
300 0.0641164779663086
310 0.06134885549545288
320 0.06123071908950806
330 0.057721227407455444
340 0.0517650730907917
350 0.050133995711803436
360 0.04808633029460907
370 0.0463901124894619
380 0.04569254070520401
390 0.04365114867687225
400 0.042057789862155914
410 0.04086320102214813
420 0.041335586458444595
430

In [11]:
pred=list()

with torch.no_grad():
  model.eval()
  y_test=model(test.to(device))
  a,b=torch.max(y_test,dim=1)
  pred.extend(b.tolist())

In [12]:
submit["label"]=pred
submit.to_csv("submit.csv",index=False)